# Mass Mobilization Project
<img style="float: right;" src="../images/man_with_hammer.png">

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import re

In [2]:
# mass = pd.read_csv('../source/mmALL_073120_csv.csv')
# mass.head(3)

In [3]:
pwd

'/Users/emilynaftalin/Data_Science/General Assembly/dsi/projects/Mass-Protests/users'

In [4]:
mass = pd.read_pickle('../data/01_mass_EDA_protestor_category.pickle')
mass.head()

,id,country,ccode,year,region,protestnumber,startday,startmonth,startyear,endday,...,stateresponse1,stateresponse2,stateresponse3,stateresponse4,stateresponse5,stateresponse6,stateresponse7,sources,notes,protester_id_type
0,201990001,Canada,20,1990,North America,1,15.0,1.0,1990.0,15.0,...,ignore,NaN,NaN,NaN,NaN,NaN,NaN,1. great canadian train journeys into history;...,canada s railway passenger system was finally ...,workers_unions
1,201990002,Canada,20,1990,North America,2,25.0,6.0,1990.0,25.0,...,ignore,NaN,NaN,NaN,NaN,NaN,NaN,1. autonomy s cry revived in quebec the new yo...,protestors were only identified as young peopl...,students_youth
2,201990003,Canada,20,1990,North America,3,1.0,7.0,1990.0,1.0,...,ignore,NaN,NaN,NaN,NaN,NaN,NaN,1. quebec protest after queen calls for unity ...,"the queen, after calling on canadians to remai...",political_group
3,201990004,Canada,20,1990,North America,4,12.0,7.0,1990.0,6.0,...,accomodation,NaN,NaN,NaN,NaN,NaN,NaN,1. indians gather as siege intensifies; armed ...,canada s federal government has agreed to acqu...,ethnic_group
4,201990005,Canada,20,1990,North America,5,14.0,8.0,1990.0,15.0,...,crowd dispersal,arrests,accomodation,NaN,NaN,NaN,NaN,1. dozens hurt in mohawk blockade protest the ...,protests were directed against the state due t...,locals_residents


In [5]:
mass.columns

Index(['id', 'country', 'ccode', 'year', 'region', 'protestnumber', 'startday',
       'startmonth', 'startyear', 'endday', 'endmonth', 'endyear',
       'protesterviolence', 'location', 'participants_category',
       'participants', 'protesteridentity', 'protesterdemand1',
       'protesterdemand2', 'protesterdemand3', 'protesterdemand4',
       'stateresponse1', 'stateresponse2', 'stateresponse3', 'stateresponse4',
       'stateresponse5', 'stateresponse6', 'stateresponse7', 'sources',
       'notes', 'protester_id_type'],
      dtype='object')

In [6]:
# mass = mass[mass['protest']==1]

In [7]:
mass.participants_category.value_counts()

100-999       3204
50-99         2508
2000-4999     1580
>10000        1470
5000-10000     642
1000-1999      483
Name: participants_category, dtype: int64

In [8]:
mass.participants_category.isnull().sum()

5352

In [9]:
mass['participants'].isnull().sum()

12

In [10]:
mass = mass[mass['participants'].notnull()]

In [11]:
mass['participants'].isnull().sum()

0

In [12]:
def string_to_number(string):

    # if nan then just exit
    if string is np.nan:
        return string
    
    output = re.findall (  r'\d+(?:,\d+)?' , string)
        
    if len(output ) == 0:
        return string
    elif len(output) == 1:
        return output[0]
    elif len(output) > 1:
        return max(output)
    else:
        return output

In [13]:
'tens of thousands'.split(' ')

['tens', 'of', 'thousands']

In [14]:

#Used to convert participants column from a mess of text into nice clean numbers
# Return value is still a string value, but can be converted easily later

def participants_to_numbers(string):

    # if nan then just exit
    if string is np.nan:
        return string
    
    if type(string) is list:  # in case list of numbers
        return string
    
    string = string.lower()
    
    small_groups = ['dozen','dozens', 'a group', 'group','several dozen','scores','people','crowd','crowds']   # 50 -99
    medium_groups = ['busloads','hundreds','widespread','several hundred','large','hundred','protests','demonstrators','teachers','.']  #100 - 999
    large_groups = ['thousand','thousands','several thousand', 'mass', 'huge'] #1000-5000
    very_large_groups = ['tens of thousands']  #5000-10_000
    huge_groups =  ['government']        #10_000 - 100_000
    massive_groups = ['hundreds of thousands','millions','million','venezuela']    # >  100_000
    
    # try to match the incoming string as itself first
    if string in small_groups:
        return 50
    elif string in medium_groups:
        return 200
    elif string in large_groups:
        return 1500
    elif string in very_large_groups:
        return 20_000
    elif string in huge_groups:
        return 50_000
    elif string in massive_groups:
        return 600_000
    
    # if the string is not found in above word groups, split it up as a list of words
    word_list = string.split(' ')
    
    word_list.reverse()   # to handle cases like hundreds of thousands that might fall through
    
    for word in word_list:
        
        word = word.replace('\"','')
        
        if word in small_groups:
            return 50
        elif word in medium_groups:
            return 200
        elif word in large_groups:
            return 1500
        elif word in very_large_groups:
            return 20_000
        elif word in huge_groups:
            return 50_000
        elif word in massive_groups:
            return 600_000
    
    return string

In [15]:
def participants_to_categories(participants):

    # In this function participants should be an int
   
    # if 0 then just exit - but this should probably never happen
    if participants == 0:
        return participants
    
    if participants > 100_000:
        return 'Over 100,000'
    elif participants >= 10_000:
        return '10,000-100,000' 
    elif participants >= 5_000:
        return '5,000-9,999'
    elif participants >= 1000:
        return '1,000-4,999'
    elif participants >= 100:
        return '100-999'
    elif participants >= 50:
        return '50-99'
    else:
        return 'Less than 50'


In [16]:

#mass['participants_converted'] = mass['participants']
mass['participants_converted'] = mass['participants'].apply(string_to_number)
mass['participants_converted'] = mass['participants_converted'].apply(participants_to_numbers)

In [17]:
# mass = mass[mass['participants'].notnull()]

mass['participants_converted'] = mass['participants_converted'].replace(',','', regex=True)

mass['participants_converted']= mass['participants_converted'].astype('int64') 

mass['participants_cat_converted'] = mass['participants_converted'].apply(participants_to_categories)


In [18]:
#mass[mass['participants_converted'] > 0]   [['country','year','participants_category','participants','participants_converted','participants_cat_converted','protesterdemand1','stateresponse1']].iloc[50:100]
mass[mass['participants_category'] != mass['participants_cat_converted']]   [['country','year','participants_category','participants','participants_converted','participants_cat_converted','protesterdemand1','stateresponse1']].iloc[6000:6050]

,country,year,participants_category,participants,participants_converted,participants_cat_converted,protesterdemand1,stateresponse1
11852,Madagascar,2004,1000-1999,1800+,1800,"1,000-4,999",labor wage dispute,accomodation
11859,Madagascar,2006,2000-4999,1000s,1000,"1,000-4,999",removal of politician,ignore
11860,Madagascar,2007,2000-4999,1000s,1000,"1,000-4,999","political behavior, process",arrests
11861,Madagascar,2007,>10000,10000+,10000,"10,000-100,000","political behavior, process",killings
11864,Madagascar,2009,>10000,10000s,10000,"10,000-100,000","political behavior, process",shootings
11866,Madagascar,2009,2000-4999,1000+,1000,"1,000-4,999","political behavior, process",shootings
11867,Madagascar,2009,2000-4999,1000+,1000,"1,000-4,999","political behavior, process",shootings
11868,Madagascar,2009,>10000,30000+,30000,"10,000-100,000","political behavior, process",shootings
11872,Madagascar,2011,1000-1999,1000,1000,"1,000-4,999","political behavior, process",crowd dispersal
11873,Madagascar,2012,2000-4999,3000-5000,5000,"5,000-9,999","political behavior, process",crowd dispersal


In [19]:
mass[ (mass['participants_category'] != mass['participants_cat_converted']) & mass['participants_category'].notna()] [['country','year','participants_category','participants','participants_converted','participants_cat_converted','protesterdemand1','stateresponse1']].iloc[150:200]

,country,year,participants_category,participants,participants_converted,participants_cat_converted,protesterdemand1,stateresponse1
1228,Colombia,2019,>10000,tens of thousands,20000,"10,000-100,000","political behavior, process",crowd dispersal
1229,Colombia,2019,>10000,tens of thousands,20000,"10,000-100,000","political behavior, process",crowd dispersal
1479,Venezuela,2014,>10000,10000,10000,"10,000-100,000","political behavior, process",crowd dispersal
1484,Venezuela,2014,5000-10000,5000,5000,"5,000-9,999","political behavior, process",crowd dispersal
1488,Venezuela,2014,>10000,10000s,10000,"10,000-100,000","political behavior, process",crowd dispersal
1490,Venezuela,2014,5000-10000,5000,5000,"5,000-9,999","price increases, tax policy",crowd dispersal
1494,Venezuela,2015,2000-4999,1000s,1000,"1,000-4,999","political behavior, process",ignore
1502,Venezuela,2016,>10000,Thousands,1500,"1,000-4,999",removal of politician,crowd dispersal
1503,Venezuela,2016,100-999,several dozen arrests,50,50-99,"price increases, tax policy",arrests
1504,Venezuela,2016,5000-10000,several thousand,1500,"1,000-4,999",removal of politician,crowd dispersal


In [20]:
mass['participants_cat_converted'].value_counts()

100-999           4746
1,000-4,999       3897
50-99             3493
10,000-100,000    2121
5,000-9,999        576
Over 100,000       369
Less than 50        25
Name: participants_cat_converted, dtype: int64

In [ ]:
# saving updated df as pickle 

In [21]:
mass.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15227 entries, 0 to 17141
Data columns (total 33 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          15227 non-null  int64  
 1   country                     15227 non-null  object 
 2   ccode                       15227 non-null  int64  
 3   year                        15227 non-null  int64  
 4   region                      15227 non-null  object 
 5   protestnumber               15227 non-null  int64  
 6   startday                    15227 non-null  float64
 7   startmonth                  15227 non-null  float64
 8   startyear                   15227 non-null  float64
 9   endday                      15227 non-null  float64
 10  endmonth                    15227 non-null  float64
 11  endyear                     15227 non-null  float64
 12  protesterviolence           15227 non-null  float64
 13  location                    152

In [30]:
mass.columns

Index(['id', 'country', 'ccode', 'year', 'region', 'protestnumber', 'startday',
       'startmonth', 'startyear', 'endday', 'endmonth', 'endyear',
       'protesterviolence', 'location', 'participants_category',
       'participants', 'protesteridentity', 'protesterdemand1',
       'protesterdemand2', 'protesterdemand3', 'protesterdemand4',
       'stateresponse1', 'stateresponse2', 'stateresponse3', 'stateresponse4',
       'stateresponse5', 'stateresponse6', 'stateresponse7', 'sources',
       'notes', 'protester_id_type', 'participants_converted',
       'participants_cat_converted'],
      dtype='object')

In [32]:
mass.drop(columns=['participants_category','participants'], inplace=True)

In [33]:
mass.columns

Index(['id', 'country', 'ccode', 'year', 'region', 'protestnumber', 'startday',
       'startmonth', 'startyear', 'endday', 'endmonth', 'endyear',
       'protesterviolence', 'location', 'protesteridentity',
       'protesterdemand1', 'protesterdemand2', 'protesterdemand3',
       'protesterdemand4', 'stateresponse1', 'stateresponse2',
       'stateresponse3', 'stateresponse4', 'stateresponse5', 'stateresponse6',
       'stateresponse7', 'sources', 'notes', 'protester_id_type',
       'participants_converted', 'participants_cat_converted'],
      dtype='object')

In [35]:
mass.rename(columns={"participants_converted": "partipants_number", "participants_cat_converted": "protest_size_category"}, inplace=True)

In [36]:
mass.columns

Index(['id', 'country', 'ccode', 'year', 'region', 'protestnumber', 'startday',
       'startmonth', 'startyear', 'endday', 'endmonth', 'endyear',
       'protesterviolence', 'location', 'protesteridentity',
       'protesterdemand1', 'protesterdemand2', 'protesterdemand3',
       'protesterdemand4', 'stateresponse1', 'stateresponse2',
       'stateresponse3', 'stateresponse4', 'stateresponse5', 'stateresponse6',
       'stateresponse7', 'sources', 'notes', 'protester_id_type',
       'partipants_number', 'protest_size_category'],
      dtype='object')

In [37]:
pwd

'/Users/emilynaftalin/Data_Science/General Assembly/dsi/projects/Mass-Protests/users'

In [38]:
mass.to_pickle('../data/01_EDA_participant_numbers.pkl')

In [39]:
mass.to_csv('../data/01_EDA_participant_numbers.csv', index=False)